Original file notes

# Calculate Arctic Regional ice mass budget terms from CESM2-LE

* **Description**: Reads in and creates regional integrated values from CESM2-LE
* **Input data**: CESM2-LE data, read using intake-esm
* **Output data**: Netcdf with processed ice state for all members
* **Creator**: Alice DuVivier
* **Date**: August 2021

I have commented comments out pop_tools here since I don't nned it.

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
# import pop_tools
from datetime import timedelta
import glob
import dask
from matplotlib.gridspec import GridSpec
import pandas as pd

import intake
from distributed import Client
from ncar_jobqueue import NCARCluster


/glade/work/jonahshaw/miniconda3/envs/cheydask/lib/python3.7/site-packages/ncar_jobqueue/cluster.py:31: UserWarning: Unable to determine which NCAR cluster you are running on... Using an instance of `distributed.LocalCluster` class.
  'Unable to determine which NCAR cluster you are running on... Using an instance of `distributed.LocalCluster` class.'


^always get this error that makes it seem like the program doesn't realize that it is actually running on Casper.

Spin up dask cluster

I have only replaced the "project" line here. It was giving me a cluster to run on a few weeks ago and I am not sure what has changed. 
I tried running both on a casper login node and a casper pbs batch with no success.
Perhaps updates to the Jupyterhub are causing issues?

In [2]:
# import dask

# Use dask jobqueue
from dask_jobqueue import PBSCluster

# Import a client
from dask.distributed import Client

# Setup your PBSCluster
cluster = PBSCluster(
    cores=2, # The number of cores you want
    memory='32 GB', # Amount of memory
    processes=1, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='$TMPDIR', # Use your local directory
    #resource_spec='select=1:ncpus=2:mem=256GB', # Specify resources
    project='UCUC0007', # Input your project ID here
    # project='P93300665', # Input your project ID here
    walltime='04:00:00', # Amount of wall time
    interface='ib0', # Interface to use
)

print(cluster)

# Scale up
cluster.scale(32)

# Change your url to the dask dashboard so you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

PBSCluster('tcp://10.12.206.63:42353', workers=0, threads=0, memory=0 B)


In [6]:
%%time
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    dsets = subset.to_dataset_dict(cdf_kwargs={'chunks': {'time':240}, 'decode_times': True})

NameError: name 'subset' is not defined